In [1]:
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise
from lib.environment.attacker.environment import Environment
import numpy as np
import os
from datetime import datetime

2024-05-10 11:37:12.002836: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 11:37:12.060908: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 11:37:13.181522: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
task_training_name = "attacker"

In [3]:
def create_env():
    def _init():
        return Environment()
    return _init

In [4]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [5]:
def get_task_models_path():
    current_datetime = datetime.now()

    year = current_datetime.year
    month = current_datetime.month
    day = current_datetime.day
    hour = current_datetime.hour
    minute = current_datetime.minute
    second = current_datetime.second

    datetime_name = f"{year}_{month}_{day}_{hour}_{minute}_{second}"

    return f"models/{task_training_name}/{datetime_name}"

In [6]:
save_path = get_task_models_path()

create_folder_if_not_exists(save_path)

In [7]:
num_threads = 8
model_filename_prefix = "DDPG_model"

In [8]:
env = SubprocVecEnv([create_env() for i in range(num_threads)])

n_actions = env.action_space.shape[-1]
n_actions

2024-05-10 11:37:25.147952: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 11:37:25.163889: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 11:37:25.188925: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 11:37:25.197587: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly 

2

In [9]:
std_dev = 0.2
tau = 0.005
gamma = 0.99
learning_rate = 0.001
policy = "MlpPolicy"

In [10]:
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

In [ ]:
model = DDPG(
    policy=policy,
    env=env,
    tau=tau,
    gamma=gamma,
    action_noise=action_noise,
    learning_rate=learning_rate,
    verbose=2)

In [ ]:
total_timesteps = 10000000

In [ ]:
saved_model_number = 5
save_freq = total_timesteps / (saved_model_number * num_threads)
log_interval = total_timesteps / 10

In [ ]:
checkpoint_callback = CheckpointCallback(
    save_freq=save_freq,
    save_path=save_path,
    name_prefix=model_filename_prefix)

In [ ]:
model.learn(
    total_timesteps=total_timesteps,
    log_interval=log_interval,
    callback=checkpoint_callback,
    progress_bar=True)